# 2. Create an Incident from an Alert

Notebook to build all of the different stix objects associated with a Phishing Alert imitialising an Incident. Note that the Stix objects created by these classes are frozen dicts, so its better to create the components and then the whole, rather than create an empty whole first and then add the components, which requires updating.

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are eight stages:

1. First setup the global parameters and retrieve the context memory
2. Create the objects from the phishing email Alert,
2. Add these objects to the `ObservedData` object
3. Then create the `Indicator` and get the user `Identity` object from context memory
4. Create the `Sighting` object with the `SightingAlert` extension
5. Create the Event and the Sequence objects
6. Create the Incident, with the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc), and add all of the objects to it
7. Save the Context Memory


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).


![](img/Evidence_Alert.png)
![](img/Evidence_Alert_Flows.png)

## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.Objects.SCO.Email_Message.make_email_msg import main as make_email_msg
from Block_Families.Objects.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block,
)
from Utilities.local_make_general import invoke_save_options_context_block, invoke_save_incident_context_block, invoke_get_from_options_block, invoke_get_from_incident_block, invoke_move_unattached_to_other_block
from conv import conv

context_base = "../Orchestration/Context_Mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com",
    "reporter": "Naive Smith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}
alert_details = {
    "alert_name": "Suspcious Email",
    "alert_source": "user-report",
    "alert_log": "I have found a suspicious email"
}

TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"

## B. Create the Objects to Document the Phishing Alert

### B.1 Create the Email Address for the Threat Actor

In [4]:
email_path = "SCO/Email_Addr/email_addr_THREAT.json"
results_path = "step1/THREAT__"
# 1. invoke the email object using method from ./Utilities/local_make_sco.py
threat_email_addr = invoke_make_email_addr_block(email_path, results_path)
# 2. add the record to the in-session bundles and incident lists
results_path2 = "__email.json"
context_type = {
    "context_type": "unattached"
}
email_object_path = results_base + results_path + results_path2
email_results_context_path = results_base + results_path + "context" + results_path2
result = invoke_save_incident_context_block(email_object_path, email_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "value": "evil@northkorea.nk",
    "display_name": "Bad Man"
}
 result->{'context_result': ' incident context saved -> unattached stix_id -> email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212'}


### B.2 Retrieve the TR User Account, the Email Address, and the TR Identity from the Context Memory

In [5]:
# 1. For Naive Smith, find the user account and the email. Plus get my own identity
reporter_name = alert_data["reporter"]
to_email_addr = alert_data["to"]
TR_name = "Trusty Jones"
#
# 2. Setup variables and queries to get the objects from options
#
context_type = {
    "context_type": "users"
}
email_query = {
    "type" : "email-addr",
    "property": {
        "path": ["value"],
        "source_value": to_email_addr,
        "comparator": "EQ"
    }
}
reporter_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": reporter_name,
        "comparator": "EQ"
    }
}
reporter_email_addr = invoke_get_from_options_block(email_query, context_type, source_value=None, source_id=None)
usr_acct_query = {
    "type" : "user-account",
    "property": {
        "path": ["id"],
        "source_value": reporter_email_addr["belongs_to_ref"],
        "comparator": "EQ"
    }
}
reporter_account = invoke_get_from_options_block(usr_acct_query, context_type, source_value=None, source_id=None)
reporter_identity = invoke_get_from_options_block(reporter_ident_query, context_type, source_value=None, source_id=None)
context_type = {
    "context_type": "me"
}
TR_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": TR_name,
        "comparator": "EQ"
    }
}
TR_identity = invoke_get_from_options_block(TR_ident_query, context_type, source_value=None, source_id=None)
#
print(f"\nreporter_email_addr-->{reporter_email_addr}")
print(f"\nreporter_account-->{reporter_account}")
print(f"\nreporter_identity-->{reporter_identity}")
print(f"\nTR_identity-->{TR_identity}")
#
# 3. save the objects to "unattached"
#
context_type = {
    "context_type": "unattached"
}
email_results_obj_path = results_base + "/step0/Usr1__email.json"
email_results_context_path = results_base + "/step1/context/usr2__email_context.json"
result = invoke_save_incident_context_block(email_results_obj_path, email_results_context_path, context_type)
print(f" result->{result}")
acct_results_obj_path = results_base + "/step0/Usr1__usr_acct.json"
acct_results_context_path = results_base + "/step1/context/usr2__usr_acct_context.json"
result = invoke_save_incident_context_block(acct_results_obj_path, acct_results_context_path, context_type)
print(f" result->{result}")
ident_results_obj_path = results_base + "/step0/Usr1__ident.json"
ident_results_context_path = results_base  + "/step1/context/usr2__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")
ident_results_obj_path = results_base  + "/step0/TR_Identity__ident.json"
ident_results_context_path = results_base + "/step1/context/TR_Identity__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")


reporter_email_addr-->{'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--4722424c-7012-56b0-84d5-01d076fc547b', 'value': 'naive@example.com', 'display_name': 'Naive Smith', 'belongs_to_ref': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b'}

reporter_account-->{'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b', 'user_id': '79563902', 'account_login': 'nsmith', 'account_type': 'sales,', 'display_name': 'Naive Smith'}

reporter_identity-->{'type': 'identity', 'spec_version': '2.1', 'id': 'identity--0ff92ca8-5425-4649-bda9-37c564921944', 'created': '2024-04-27T06:06:06.901Z', 'modified': '2024-04-27T06:06:06.901Z', 'name': 'Naive Smith', 'description': 'A Naive Individual', 'roles': ['user', 'sales'], 'identity_class': 'individual', 'sectors': ['technology'], 'extensions': {'extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498': {'extension_type': 'property-extension', 'contact_numbers': [{'contact_number_typ

### B.3 Make the URL

In [6]:
url_path = "SCO/URL/suspicious_url.json"
results_path = "step1/THREAT__url.json"
hyperlink = alert_data["url"]
# make the email object
sus_url = invoke_make_url_block(url_path, results_path, hyperlink)
# add the record to the in-session bundles and lists+
context_type = {
    "context_type": "unattached"
}
url_results_context_path = results_base + "/step1/THREAT__url_context.json"
sus_url_context = invoke_save_incident_context_block(results_base + results_path, url_results_context_path, context_type)
print(f" sus_url_context->{sus_url_context}")

{
    "type": "url",
    "spec_version": "2.1",
    "id": "url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7",
    "value": "https://www.northkorea.nk/we/are/mad/"
}
 sus_url_context->{'context_result': ' incident context saved -> unattached stix_id -> url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7'}


### B.4 Make the Email Message

In [7]:
msg_path = "SCO/Email_Message/suspicious_email_msg.json"
results_path = "step1/SUSS__email_msg.json"
from_ref = threat_email_addr
to_ref = [reporter_email_addr]
cc_ref = []
bcc_ref = []
# make the email object
sus_msg = invoke_make_e_msg_block(msg_path, results_path, from_ref, to_ref, cc_ref, bcc_ref)
# add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
msg_results_context_path = results_base + "step1/THREAT__email_msg_context.json"
result = invoke_save_incident_context_block(results_base + results_path, msg_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "email-message",
    "spec_version": "2.1",
    "id": "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
    "is_multipart": false,
    "date": "2020-10-19T01:01:01Z",
    "from_ref": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "to_refs": [
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b"
    ],
    "subject": "we are coming for you",
    "body": "some bad stuff written here"
}
 result->{'context_result': ' incident context saved -> unattached stix_id -> email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d'}


### B.4 Make the "derived-from" SRO

In [8]:
# make the email object
relationship_type = "derived-from"
sro_data_path = "SRO/Relationship/sro_derived.json"
sro_results_path = "step1/SRO_derived"
sus_relation = invoke_sro_block(sro_data_path, sro_results_path, sus_msg, sus_url, relationship_type)
sro_results_path2 = results_base + sro_results_path + "__rel.json"
# add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
msg_results_context_path = results_base + "step1/context/THREAT__email_SRO_context.json"
result = invoke_save_incident_context_block(sro_results_path2, msg_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "relationship",
    "spec_version": "2.1",
    "id": "relationship--42ad1727-8b28-4556-a1f6-e4d79a4119b7",
    "created": "2024-04-27T06:08:50.943Z",
    "modified": "2024-04-27T06:08:50.943Z",
    "relationship_type": "derived-from",
    "source_ref": "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
    "target_ref": "url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7"
}
 result->{'context_result': ' incident context saved -> unattached stix_id -> relationship--42ad1727-8b28-4556-a1f6-e4d79a4119b7'}


## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [9]:
# 1. Emulate "Add to List" Block
observed_obj = []
observed_obj.append(threat_email_addr)
observed_obj.append(reporter_account)
observed_obj.append(reporter_email_addr)
observed_obj.append(sus_url)
observed_obj.append(sus_msg)
observed_obj.append(sus_relation)
# 2. Setup path to form and results
obs_path ="SDO/Observed_Data/observation-alert.json"
results_path ="step1/observation-alert.json"
# 3. Invoke the Make Observed Data Block
obs_1 = invoke_make_observed_data_block(obs_path, results_path, observation=observed_obj)
# 4. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
obs_results_context_path = results_base + "step1/observation-alert_context.json"
result = invoke_save_incident_context_block(results_base + results_path, obs_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--7e6e1eb1-72a1-437a-9f98-9533a6215901",
    "created": "2024-04-27T06:08:50.976Z",
    "modified": "2024-04-27T06:08:50.976Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
        "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b",
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b",
        "url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7",
        "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
        "relationship--42ad1727-8b28-4556-a1f6-e4d79a4119b7"
    ]
}
 result->{'context_result': ' incident context saved -> unattached stix_id -> observed-data--7e6e1eb1-72a1-437a-9f98-9533a6215901'}


## D. Create the Alert Sighting

Tbis then represents the sighting of potential phishing email as a group of elements

### D.1 Create a Indicator with a Pattern for the Phishing Email
but set its status to `unkown` to start with

In [10]:
# 1. Setup pattern for indicator
pat1 = "[email-addr:value = '" + threat_email_addr["value"] + "' AND email:subject = '" + sus_msg["subject"] + "']"
# 2. Setup path to form and results
ind_path ="SDO/Indicator/indicator_alert.json"
results_path ="step1/indicator_alert.json"
# 3. Invoke the Make Observed Data Block
ind_1 = invoke_make_indicator_block(ind_path, results_path, pattern=pat1)
# 4. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
indicator_results_context_path = results_base + "step1/indicator_context.json"
result = invoke_save_incident_context_block(results_base + results_path, indicator_results_context_path, context_type)
print(f" result->{result}")

inputfile->../Block_Families/Objects/SDO/Indicator/indicator_alert.json
path exists
input->{'indicator_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': []}, 'object': {'name': 'Potential Phishing Email', 'description': 'Suspicious email reported by user', 'pattern_type': 'stix', 'pattern': '', 'indicator_types': ['unknown']}, 'extensions': {}, 'sub': {}}, 'pattern': "[email-addr:value = 'evil@northkorea.nk' AND email:subject = 'we are coming for you']"}
pattern [email-addr:value = 'evil@northkorea.nk' AND email:subject = 'we are coming for you']
{
    "type": "indicator",
    "spec_version": "2.1",
    "id": "indicator--5434d6c3-7f15-45f4-8fd3-f603c4676a28",
    "created": "2024-04-27T06:08:51.230Z",
    "modified": "2024-04-27T06:08:51.230Z",
    "name": "Potential Phi

### D.2 Create the Sighting
Connecting the:
- observed-data object
- indicator object
- the reporter identity as the location

In [11]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_alert.json"
results_path ="step1/sighting_alert.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighted = ind_1
observed_list = [obs_1]
where_list = [reporter_identity]
# 2. Invoke the Make Observed Data Block
sight1 = invoke_sighting_block(sighting_data_path, results_path, observed=observed_list, sighted=sighted, where=where_list)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
sighting_results_context_path = results_base + "step1/sighting_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sighting_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--2db052b6-4944-449a-a87a-c224993e85ba",
    "created": "2024-04-27T06:08:51.660Z",
    "modified": "2024-04-27T06:08:51.660Z",
    "count": 1,
    "sighting_of_ref": "indicator--5434d6c3-7f15-45f4-8fd3-f603c4676a28",
    "observed_data_refs": [
        "observed-data--7e6e1eb1-72a1-437a-9f98-9533a6215901"
    ],
    "where_sighted_refs": [
        "identity--0ff92ca8-5425-4649-bda9-37c564921944"
    ],
    "extensions": {
        "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43": {
            "extension_type": "property-extension"
        },
        "sighting-alert": {
            "name": "user-report",
            "log": "I have found a suspicious email",
            "system_id": "XYZ-1234",
            "source": "user submission"
        }
    }
}
==== sighting--2db052b6-4944-449a-a87a-c224993e85ba
 result->{'context_result': ' incident context saved -> unattached stix_id -> sighting--2db052b6-4944-4

### D.3 Copy Everything from Unattached to Other

Everything that uis connected in the Sighting should now be moved Unattached to the Other list

In [12]:
total_list = observed_obj
total_list.append(obs_1)
total_list.append(ind_1)
total_list.append(sight1)
total_list.append(reporter_identity)

obs_context_move_path = results_base + "step1/context/context_move.json"
obs_context_move_results = results_base + "step1/context/context_move_results.json"
result = invoke_move_unattached_to_other_block(obs_context_move_path, obs_context_move_results, total_list)

delete element 0, node id email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212
delete element 1, node id user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b
delete element 0, node id email-addr--4722424c-7012-56b0-84d5-01d076fc547b
delete element 2, node id url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7
delete element 2, node id email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d
delete element 2, node id observed-data--7e6e1eb1-72a1-437a-9f98-9533a6215901
delete element 2, node id indicator--5434d6c3-7f15-45f4-8fd3-f603c4676a28
==== sighting--2db052b6-4944-449a-a87a-c224993e85ba
delete element 0, node id identity--0ff92ca8-5425-4649-bda9-37c564921944


## E. Create the Event Objects
Tbis Event represents the receiving of the phishing email

In [13]:
# 1. Setup path to form and results
event_data_path ="SDO/Event/event_alert.json"
results_path ="step1/event_alert.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighting = sight1["id"]
# 2. Invoke the Make Observed Data Block
event_1 = invoke_make_event_block(event_data_path, results_path, changed_objects=None,sighting_refs=sighting)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "event"
}
event_results_context_path = results_base + "step1/context/event_context.json"
result = invoke_save_incident_context_block(results_base + results_path, event_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "event",
    "spec_version": "2.1",
    "id": "event--a985faf5-096a-4b8f-84de-07d4d63903a3",
    "created": "2024-04-27T06:08:51.365Z",
    "modified": "2024-04-27T06:08:51.365Z",
    "description": "Suspicious email reported by user",
    "end_time": "2024-04-27T06:08:51.365Z",
    "event_types": [
        "dissemination-phishing-emails"
    ],
    "name": "Potential Phishing Email",
    "start_time": "2024-04-27T06:08:51.365Z",
    "extensions": {
        "extension-definition--4ca6de00-5b0d-45ef-a1dc-ea7279ea910e": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> event stix_id -> event--a985faf5-096a-4b8f-84de-07d4d63903a3'}



### E.2 Then Create the Sequence Object for the Event

The Sequence object is created to place the event in a sequence, in this case a single-step

In [14]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_event_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "event"
sequenced_object = event_1["id"]
# 2. Invoke the Make Observed Data Block
seq_E_1 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "sequence"
}
sequence_results_context_path = results_base + "step1/event_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sighting_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--5756643e-0060-4e82-a123-3d121f27cbb4",
    "created": "2024-04-27T06:08:51.412Z",
    "modified": "2024-04-27T06:08:51.412Z",
    "sequenced_object": "event--a985faf5-096a-4b8f-84de-07d4d63903a3",
    "sequence_type": "event",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> sequence stix_id -> sequence--5756643e-0060-4e82-a123-3d121f27cbb4'}



### E.3 Since it is the First Sequence, Create the Start Sequence Object and Connect it

The Sequence object is created to place the event in a sequence, in this case a start-step

In [15]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_start.json"
results_path ="step1/sequence_event_start.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "start_step"
sequence_type = "event"
on_completion = seq_E_1["id"]
# 2. Invoke the Make Observed Data Block
seq_E_0 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=None, on_completion=on_completion, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "start"
}
event_results_context_path = results_base + "step1/event_seq_start_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sighting_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--868b0dfb-bb9c-43bb-9d50-48686e0ae9d0",
    "created": "2024-04-27T06:08:51.460Z",
    "modified": "2024-04-27T06:08:51.460Z",
    "sequence_type": "event",
    "step_type": "start_step",
    "on_completion": "sequence--5756643e-0060-4e82-a123-3d121f27cbb4",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> start stix_id -> sequence--868b0dfb-bb9c-43bb-9d50-48686e0ae9d0'}


## F. Create the Task Objects

Tbis Task represents the first Task to clarify the situation first,no real Impactyet

In [16]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_alert.json"
results_path ="step1/task_alert.json"
# 2. Invoke the Make Observed Data Block
task_1 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "task"
}
task_results_context_path = results_base + "step1/task_context.json"
result = invoke_save_incident_context_block(results_base + results_path, task_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--f4384099-72ba-4758-97f0-7506598fb151",
    "created": "2024-04-27T06:08:51.504Z",
    "modified": "2024-04-27T06:08:51.504Z",
    "description": "Suspicious email reported by user",
    "end_time": "2024-04-27T06:08:51.504Z",
    "name": "Potential Phishing Email",
    "start_time": "2024-04-27T06:08:51.504Z",
    "extensions": {
        "extension-definition--2074a052-8be4-4932-849e-f5e7798e0030": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> task stix_id -> task--f4384099-72ba-4758-97f0-7506598fb151'}


### F.2 Then Create the Sequence Object for the Task

The Sequence object is created to place the Task in a sequence, in this case a single-step

In [17]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_task_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_1["id"]
# 2. Invoke the Make Observed Data Block
seq_T_1 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "sequence"
}
event_sequence_results_context_path = results_base + "step1/task_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, event_sequence_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--fe62dcc2-fa74-4285-baa1-f2bf96286dc2",
    "created": "2024-04-27T06:08:51.553Z",
    "modified": "2024-04-27T06:08:51.553Z",
    "sequenced_object": "task--f4384099-72ba-4758-97f0-7506598fb151",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> sequence stix_id -> sequence--fe62dcc2-fa74-4285-baa1-f2bf96286dc2'}



### F.3 Then Create the Sequence Start Object for the Tasks

The Sequence object is created to place the Task in a sequence, in this case a start-step

In [18]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_start.json"
results_path ="step1/sequence_task_start.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "start_step"
sequence_type = "task"
on_completion = seq_T_1["id"]
# 2. Invoke the Make Observed Data Block
seq_T_0 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=None, on_completion=on_completion, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "start"
}
task_seq_results_context_path = results_base + "step1/task_seq_start_context.json"
result = invoke_save_incident_context_block(results_base + results_path, task_seq_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--7304d6c1-fe6c-4fb9-8b23-906097976e0f",
    "created": "2024-04-27T06:08:51.601Z",
    "modified": "2024-04-27T06:08:51.601Z",
    "sequence_type": "task",
    "step_type": "start_step",
    "on_completion": "sequence--fe62dcc2-fa74-4285-baa1-f2bf96286dc2",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> start stix_id -> sequence--7304d6c1-fe6c-4fb9-8b23-906097976e0f'}


## G. Finally, Create the Incident Object and Append all of the objects to it

Create the Incident object and assign all of the objets to it

In [19]:
# 1. Setup path to form and results
inc_path ="SDO/Incident/phishing_incident.json"
results_path ="step1/incident_alert.json"
# 2. Setup the Sequence Object for the Event
#
sequence_start_refs = []
sequence_refs = []
task_refs = []
event_refs = []
impact_refs = []
other_object_refs = []
# 2. Invoke the Make Observed Data Block
incident_list = invoke_make_incident_block(inc_path, results_path, sequence_start_refs, sequence_refs, task_refs,event_refs, impact_refs, other_object_refs)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "incident"
}
incident_results_context_path = results_base + "step1/incident_context.json"
result = invoke_save_incident_context_block(results_base + results_path, incident_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "incident",
    "spec_version": "2.1",
    "id": "incident--456239b6-ff83-4c14-8d40-c8295e5fb60c",
    "created": "2024-04-27T06:08:51.648Z",
    "modified": "2024-04-27T06:08:51.648Z",
    "name": "potential phishing",
    "extensions": {
        "extension-definition--ef765651-680c-498d-9894-99799f2fa126": {
            "extension_type": "property-extension",
            "investigation_status": "new",
            "incident_types": [
                "dissemination-phishing-emails"
            ]
        }
    }
}
 result->{'context_result': ' incident context saved -> incident stix_id -> incident--456239b6-ff83-4c14-8d40-c8295e5fb60c'}


## H. Write out the Contex Memory for the Incident

Export out the Context Memory for the Incident

In [20]:
# Save the Tpe Refinery Context Memory File